<a href="https://colab.research.google.com/github/AryamanJaggi/Deception-Detection-with-Transformers/blob/main/MultiModal_Deception_Detection_using_MBT_(cleaned_for_publication).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
torch.cuda.memory_allocated(0)

torch.cuda.get_device_properties(0)



41535024128

In [ ]:
video = torch.load('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/DeceptiveFERTensors/trial_lie_001.pt', map_location=torch.device('cpu'))
print(video.shape)

permuted = torch.permute(video, (1,0))#switches embedding and time dimension aka T,F to F,T
normalizedVideoFeatures = torch.nn.functional.avg_pool1d(permuted, math.ceil(fps/10))#because fps varies grossly across the dataset (10,25,29.97,30 fps) I try to average the features so they are close to 10 fps before I padd them to match the audio

unusedvideo,audio,metadata = read_video('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Deceptive/trial_lie_001.mp4')
fps = metadata['video_fps']
audioSampleRate = metadata['audio_fps']
data = cv2.VideoCapture('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Deceptive/trial_lie_001.mp4')
frames = data.get(cv2.CAP_PROP_FRAME_COUNT)
print('frames: ' + str(frames))
seconds = frames/float(fps)
print("seconds:" + str(seconds))

print(fps)
print(audioSampleRate)

print(audio.shape)
audio = torchaudio.functional.resample(audio, audioSampleRate, 16000)
print(audio.shape)
audio = torch.mean(audio, dim=0)#takes mean and gets rid of channels dimesnion
print(audio.shape)
audio = audio.unsqueeze(0)
video = video.unsqueeze(0)


mainModel(video,audio)



torch.Size([510, 1408])


NameError: ignored

In [ ]:
#device = 'cuda'
#model_name='enet_b2_8'
#fer=HSEmotionRecognizer(model_name=model_name,device=device)
#mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
#test_transforms = transforms.Compose(
#                [
#                    transforms.Resize((fer.img_size,fer.img_size)),
#                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                                std=[0.229, 0.224, 0.225])
#                ]
#            )

for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful'): # deceptive
  name = os.path.join('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/TruthfulFERTensors',filename.replace('.mp4','.pt'))
  print(name)
  if(os.path.exists(name)):
      print("already copied previously")
      continue

  video,audio,metadata = (read_video(os.path.join('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful',filename)))#video in shape T,H,W,C
  del audio, metadata
  gc.collect()
  T,H,W,C = video.shape
  #print("video.shape: ")
  #print(video.shape)
  #print("video name: " + filename)
  #print("="*20)
  #FERFeatures = torch.empty(T, 1408)
  split_size = math.ceil(T/100.0)
  listOfTensors = video.split(split_size)
  del video
  gc.collect()
  for i in range(len(listOfTensors)):
    TensorBatch = torch.squeeze(torch.from_numpy(videoEmbeddingExtraction(torch.unsqueeze(listOfTensors[i],0))))

    if (i == 0):
      FERFeatures = TensorBatch
      continue
    if (len(TensorBatch.shape) < 2):
      TensorBatch = torch.unsqueeze(TensorBatch,0)
    #print(listOfTensors[i])
    #print(TensorBatch.shape)
    #print(FERFeatures.shape)
    FERFeatures = torch.cat((FERFeatures, TensorBatch))
    del TensorBatch
    gc.collect()

  torch.save(FERFeatures, name)
  torch.cuda.empty_cache()
  del FERFeatures
  gc.collect()


In [ ]:
device = 'cuda'
model_name='enet_b2_8'
fer=HSEmotionRecognizer(model_name=model_name,device=device)
mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
test_transforms = transforms.Compose(
                [
                    transforms.Resize((fer.img_size,fer.img_size)),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225])
                ]
            )

/root/.hsemotion/enet_b2_8.pt Compose(
    Resize(size=(260, 260), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [ ]:
#debugging Code to see all tensors stored in
#res = []
#for obj in gc.get_objects():
#  if torch.is_tensor(obj):
#    print(obj.size())
#    res.append(np.prod(obj.size()))
  # elif (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
  #   res.append(np.prod(obj.data.size()))

In [ ]:
video,audio,metadata = (read_video('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Deceptive/trial_lie_001.mp4'))#video in shape T,H,W,C
del audio, metadata
gc.collect()
T,H,W,C = video.shape
#FERFeatures = torch.empty(T, 1408)
split_size = math.ceil(T/1000.0)
listOfTensors = video.split(1)
del video
gc.collect()
FERFeatures = []
for i in range(len(listOfTensors)):
  print('='*20)
  TensorBatch = videoEmbeddingExtraction(torch.unsqueeze(listOfTensors[i],0))
  print(torch.cuda.memory_allocated(0)/1e9)
  FERFeatures.append(TensorBatch)


# name = '/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/DeceptiveFERTensors/trial_lie_001.pt'
# torch.save(FERFeatures, name)
# torch.cuda.empty_cache()
# del FERFeatures
# gc.collect()

In [ ]:
import sys
def videoEmbeddingExtraction (video): # manually implementing some of the stuff from HSME

    #video dimensions are B T H W C
    # dimensions for each frame (which extract face requires) B'(B*T) H W C
    B,T,H,W,C = video.shape
    #cv2_imshow(torch.squeeze(torch.squeeze(video[0,0,:,:,:])).cpu().numpy())
    #mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
    def extract_face(frame):
        bounding_boxes, probs = mtcnn.detect(frame, landmarks=False)
        #print("frame.shape")
        #print(frame.shape)
        #print("bounding_boxes.shape:")
        #print(bounding_boxes.shape)
        #$$$$$$4print('1:',torch.cuda.memory_allocated(0)/1e9)
        #print(bounding_boxes.shape)
        #bounding_boxes = bounding_boxes.astype(np.float64)
        #bounding_box= torch.take_along_dim(torch.from_numpy(bounding_boxes), torch.from_numpy(np.argmax(probs,1)),1)
        bounding_boxes2 = []
        #bounding_boxes = np.vstack(bounding_boxes[:]).astype(np.float64)
        #print(bounding_boxes.shape)
        #bounding_boxes = bounding_boxes.astype(np.float64)
        for i in range (bounding_boxes.shape[0]):
          indexOfBestFace = np.argmax(probs[i])
          arrayOfBoundingBoxesForOneFrame = bounding_boxes[i]
          #print(i)
          if arrayOfBoundingBoxesForOneFrame is None:
            print(name)
            bounding_boxes2.append(np.array([1000000,0,0,0]))
            continue
          #print("\narrayOfBoundingBoxesForOneFrame shape:")
          #print(arrayOfBoundingBoxesForOneFrame.shape)
          #print("specific bounding box type:" + arrayOfBoundingBoxesForOneFrame.type)
          bounding_boxes2.append(arrayOfBoundingBoxesForOneFrame[indexOfBestFace])#originalbounding_boxes2.append(bounding_boxes[i, indexOfBestFact,:])

        del indexOfBestFace, arrayOfBoundingBoxesForOneFrame, bounding_boxes
        gc.collect()
        #$$$$$$$print('2:',torch.cuda.memory_allocated(0)/1e9)
        bounding_boxes2 = np.stack(bounding_boxes2)
        bounding_boxes2 = torch.from_numpy(bounding_boxes2.astype('int32'))#.int()
        #box = bounding_box.int()
        batchlist = []
        i = 0
        height = 0
        width = 0
        for batch in range(bounding_boxes2.shape[0]):
          i += 1
          x1,y1,x2,y2=bounding_boxes2[batch,0:4]
          if (x1 == 1000000 or x1 < 0 or x2 < 0 or y1 < 0 or y2 < 0):
            batchlist.append(torch.zeros((100,100,3)))
            i -= 1
            continue

          face_img=frame[batch, y1:y2,x1:x2,:] # batch, h, w, c
          #print(face_img.shape)
          batchlist.append(face_img)
          height += face_img.shape[0]
          width += face_img.shape[1]

        if (i==0): # edge case for trial_lie_005
          return torch.zeros(1, bounding_boxes2.shape[0], 1408)

        averageHeigth = int(height/i)
        averageWidth = int(width/i)
        #$$$$$$$print('3:',torch.cuda.memory_allocated(0)/1e9)
        #del bounding_boxes2
        #gc.collect()

        resize = transforms.Compose([transforms.Resize((averageHeigth,averageWidth),interpolation=InterpolationMode.BILINEAR, max_size=None, antialias=True)])

        for j in range(bounding_boxes2.shape[0]):#i should be equal to bounding_boxes2.shape[0]
          if(torch.max(batchlist[j]).numpy() == 0):
            print("hit")
            batchlist[j] = torch.zeros(3,averageHeigth, averageWidth)
            continue
          batchlist[j] = resize(torch.permute(batchlist[j], (2,0,1)))#swithcing from hwc to chw and then resizing image
        #print('4:',torch.cuda.memory_allocated(0)/1e9)
        del bounding_boxes2
        gc.collect()
        return torch.stack(batchlist)

    facialRecognitionBatch  = video.reshape(B*T, H, W, C) #got rid of time dimension from input
    del video
    gc.collect()
    #print(facialRecognitionBatch.shape)
    #permuted = torch.permute(facialRecognitionBatch, (0, 2,3,1))# orignally in C H W for pytorch, after permute in H W C for openCV
    #cv2_imshow(torch.squeeze(facialRecognitionBatch[0,:,:,:]).cpu().numpy())

    #facialRecognitionBatchNumpy = facialRecognitionBatch.cpu().numpy()
    #facialRecognitionBatchNumpyBGR = facialRecognitionBatchNumpy[:,:,:,::-1]
    #tensorBatch = torch.from_numpy(facialRecognitionBatchNumpyBGR)
    #red, green, blue = torch.split(facialRecognitionBatch,3,3)#converting to BGR for open cv
    #facialRecognitionBatch_BGR = torch.cat(3, [blue,green,red])
    #cv2_imshow(torch.squeeze(facialRecognitionBatch_BGR[0,:,:,:]).cpu().numpy())
    face = extract_face(facialRecognitionBatch) # B C H W

    if (len(face.shape) == 3): #for edge case in trial_lie_005
      return face.cpu().numpy()

    del facialRecognitionBatch
    gc.collect()

    torch.cuda.empty_cache()

    #model_name='enet_b2_8'
    #fer=HSEmotionRecognizer(model_name=model_name,device=device)
    #features=fer.extract_features(face)
    #print(features)

    #test_transforms = transforms.Compose(
    #            [
    #                transforms.Resize((fer.img_size,fer.img_size)),
    #                transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                                            std=[0.229, 0.224, 0.225])
    #            ]
    #        )

    #face = torch.permute(face, (0,3,1,2))
    img_tensor = test_transforms(face.double())
    #print('5:',torch.cuda.memory_allocated(0)/1e9)
    del face
    gc.collect()

    features = fer.model(img_tensor.to(fer.device).float())
    #print('6:',torch.cuda.memory_allocated(0)/1e9)
    features = features.data.cpu().numpy()
    #print('7:',torch.cuda.memory_allocated(0)/1e9)
    return features

In [ ]:
for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful'): # truthful
  video,audio,metadata = (read_video(os.path.join('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful',filename)))#video in shape T,H,W,C
  del audio,metadata
  gc.collect()
  T,H,W,C = video.shape()
  #FERFeatures = torch.empty(T, 1408)
  split_size = math.ceil(T/6.0)
  listOfTensors = video.split(split_size)
  for i in range(len(listOfTensors)):
    TensorBatch = torch.squeeze(videoEmbeddingExtraction(torch.unsqueeze(listOfTensors[i])))
    del listOfTensors[i]
    gc.collect()
    if (i == 0):
      FERFeatures = TensorBatch
      continue
    FERFeatures = torch.cat((FERFeatures, TensorBatch))

  name = os.path.join('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/TruthfulFERTensors',filename.replace('.mp4','.pt'))
  torch.save(FERFeatures, name)
  torch.cuda.empty_cache()
  del FERFeatures
  gc.collect()

In [ ]:
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(device)
#model_name='enet_b2_8'

#fer=HSEmotionRecognizer(model_name=model_name,device=device)
#print(fer)
#summary(fer.model,(224,224))
#features=fer.extract_features()

cuda


In [ ]:


for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful'): # truthful

video, audio, labels = dataset.__getitem__(10)
print(video.shape)
print(audio.shape)
#frame = video[0]
#print(frame.shape)
#frame = frame.unsqueeze(0)
video = video.unsqueeze(0)
audio = audio.unsqueeze(0)
print(video.shape)
print(audio.shape)
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

torch.cuda.empty_cache()


#video, audio = video.to('cuda'), audio.to('cuda')
#print(video.is_cuda)
#print(audio.is_cuda)

#video = video.cpu().detach().numpy()
#audio = audio.cpu().detach().numpy()
#outputs = mainModel(video, audio)

In [ ]:
#will have to delete but keeping for now
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

#Do not forget to run pip install facenet-pytorch
from facenet_pytorch import MTCNN

mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)

def extract_face(frame):
    bounding_boxes, probs = mtcnn.detect(frame, landmarks=False)
    #bounding_boxes = bounding_boxes.astype(np.float64)
    #bounding_box= torch.take_along_dim(torch.from_numpy(bounding_boxes), torch.from_numpy(np.argmax(probs,1)),1)
    bounding_boxes2 = []
    bounding_boxes = bounding_boxes.astype(np.float64)
    for i in range (bounding_boxes.shape[0]):
      indexOfBestFact = np.argmax(probs[i])
      bounding_boxes2.append(bounding_boxes[i, indexOfBestFact,:])

    bounding_boxes2 = np.stack(bounding_boxes2)
    bounding_boxes2 = torch.from_numpy(bounding_boxes2).int()
    #box = bounding_box.int()
    batchlist = []
    for batch in range(bounding_boxes2.shape[0]):
      x1,y1,x2,y2=bounding_boxes2[batch,0:4]
      face_img=frame[batch, y1:y2,x1:x2,:]
      batchlist.append(face_img)
    return torch.stack(batchlist)
img = torchvision.io.read_image('/content/istockphoto-1296158947-612x612.jpg')
img = img.unsqueeze(0)
permuted = torch.permute(img, (0, 2,3,1))# orignally in C H W for pytorch, after permute in H W C for openCV
face = extract_face(permuted)
plt.imshow(face[0])
print(use_cuda)
model_name='enet_b2_8'
fer=HSEmotionRecognizer(model_name=model_name,device=device)
#features=fer.extract_features(face)
#print(features)

test_transforms = transforms.Compose(
            [
                transforms.Resize((fer.img_size,fer.img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
            ]
        )

print(face.shape)
face = torch.permute(face, (0,3,1,2))
print(face.shape)
img_tensor = test_transforms(face.double())
features = fer.model(img_tensor.to(fer.device).float())
print(features.data.max())
print(features.shape)




RuntimeError: ignored

In [ ]:
for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips'):
    if filename.endswith('.mp4'):
        my_clip = mp.VideoFileClip('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips/'+filename)
        my_clip.audio.write_audiofile('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips/'+filename+".wav")
        #subprocess.call(['ffmpeg', '-i', filename, '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '2', filename.replace('.mp4', '.wav')])#changed sampling rate from 44100
        #to 16000 becasue that is the sampling rate on which wav2vec2 was trained on and therefor had to downsample audio

for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips'):
    if filename.endswith('.mp4'):

        my_clip = mp.VideoFileClip('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips/'+filename)
        my_clip.audio.write_audiofile('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips/'+filename+".wav")
        #subprocess.call(['ffmpeg', '-i', filename, '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '2', filename.replace('.mp4', '.wav')])#changed sampling rate from 44100
        #to 16000 becasue that is the sampling rate on which wav2vec2 was trained on and therefor had to downsample audio

In [ ]:
import soundfile as sf



for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips'):
    if filename.endswith('.wav'):
        y, sr = librosa.load('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips/'+filename, sr=16000)
        #downsampling data to fit Wav2Vec2
        sf.write('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips/'+filename, y, sr, subtype='PCM_24')
        #my_clip = mp.VideoFileClip(y)
        #my_clip.audio.write_audiofile('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/DeceptiveAudioClips/'+filename)

for filename in os.listdir('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips'):
    if filename.endswith('.wav'):
        y, sr = librosa.load('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips/'+filename, sr=16000)
        #downsampling data to fit Wav2Vec2
        sf.write('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips/'+filename, y, sr, subtype='PCM_24')
        #my_clip = mp.VideoFileClip(y)
        #my_clip.audio.write_audiofile('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Audio Clips/TruthfulAudioClips/'+filename)


In [ ]:
#start of actualy notebook
!git clone https://github.com/HSE-asavchenko/face-emotion-recognition.git


fatal: destination path 'face-emotion-recognition' already exists and is not an empty directory.


In [ ]:
!pip install hsemotion
!pip install hsemotion-onnx
!pip install torch torchvision -U
!pip install Cython
! pip install facenet-pytorch
! pip install timm==0.6.13
! pip install ffmpeg
! pip install ffmpeg moviepy
!pip install librosa
!pip install transformers sentencepiece
!pip install av

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
  Created wheel for hsemotion: filename=hsemotion-0.3.0-py3-none-any.whl size=11242 sha256=fc8062a17a85708d83332868df881790cb2428e7b6dad0b4b5680d5e85022c6b
  Stored in directory: /root/.cache/pip/wheels/38/88/e0/3b365122443c2ec55f3e058f2b7ad59df7b5e302c457c4539a
Successfully built hsemotion
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.6 MB/s eta 0:00:00
  Created wheel for hsemotion-onnx: filename=hsemotion_onnx-0.3.1

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torch
import torch.nn as nn
import timm
#from timm.models.layers import conv2d_same
#timm.models.layers.conv2d_same
from hsemotion.facial_emotions import HSEmotionRecognizer
import subprocess # for ffmpeg
import moviepy.editor as mp
import librosa
import torchvision
from torchvision import transforms
#Do not forget to run pip install facenet-pytorch
from facenet_pytorch import MTCNN
from torchvision.io import read_video
import torchaudio
from transformers import Wav2Vec2Processor, SpeechEncoderDecoderModel, AutoFeatureExtractor, Wav2Vec2Model
#from torchsummary import summary
from torch.utils.data import Dataset, DataLoader

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import av

#imported specifcally for the Wav2Vec2 Feature Extrators
from transformers import Wav2Vec2FeatureExtractor
import torchaudio

from torchvision.transforms.functional import _interpolation_modes_from_int, InterpolationMode
import gc

import math
%matplotlib inline

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Deceptive . #deceptive
!cp -r /content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips/Truthful . #truthful

In [ ]:
# creates dataset used for training
class CustomDeceptionDataset(Dataset):
    def __init__(self, rootdir):
        self.deceptiveClipsPath = os.path.join(rootdir, 'Deceptive')
        self.truthfulClipsPath = os.path.join(rootdir, 'Truthful')
        self.numTruthful = len(os.listdir(self.truthfulClipsPath))-10 #subtract 10 because I want to preserve 20 of the videos for validation/testing
        self.numDeceptive = len(os.listdir(self.deceptiveClipsPath))-10 #subtract 10 because I want to preserve 20 of the videos for validation/testing
        self.deceptiveTensorsPath = os.path.join(rootdir,'DeceptiveFERTensors')
        self.truthfulTensorsPath = os.path.join(rootdir,'TruthfulFERTensors')

    def __len__(self):
        return self.numTruthful+self.numDeceptive

    def __getitem__(self, idx):
      if (idx < self.numTruthful):
        name = "trial_truth_%03d.mp4"%(idx+1)
        name2 = os.path.join(self.truthfulTensorsPath, name.replace('mp4','pt'))
        name = os.path.join(self.truthfulClipsPath, name)
        torchclass = torch.tensor([0]) # 0 for truthful

      else:
        name = "trial_lie_%03d.mp4"%((idx-self.numTruthful)+1)
        name2 = os.path.join(self.deceptiveTensorsPath, name.replace('mp4','pt'))
        name = os.path.join(self.deceptiveClipsPath, name)
        torchclass = torch.tensor([1]) # 1 for deceptive

      #my_clip = mp.VideoFileClip(name)
      #my_audio = my_clip.audio


      unusedvideo,audio,metadata = read_video(name)
      fps = metadata['video_fps']
      audioSampleRate = metadata['audio_fps']

      audio = torchaudio.functional.resample(audio, audioSampleRate, 16000)
      audio = torch.mean(audio, dim=0)#takes mean and gets rid of channels dimesnion
      video = torch.load(name2, map_location=torch.device('cpu'))

      permuted = torch.permute(video, (1,0))#switches embedding and time dimension aka T,F to F,T
      normalizedVideoFeatures = torch.nn.functional.avg_pool1d(permuted, math.ceil(fps/10))#because fps varies grossly across the dataset (10,25,29.97,30 fps) I try to average the features so they are close to 10 fps before I padd them to match the audio
      averagedVideo = torch.permute(normalizedVideoFeatures, (1,0))
      #print(name)
      #video = video.unsqueeze(0)#adding batch dimension #Im actually pretty sure your not supposed to add the batch dimension and instead its automatically added by the dataloader
      #audio = audio.unsqueeze(0)#adding batch dimension

      del normalizedVideoFeatures, permuted, fps, audioSampleRate, unusedvideo
      gc.collect()

      return averagedVideo,audio,torchclass

In [ ]:
# custom dataset containing 10 lying and 10 truthful videos used for training
class TestDataset(Dataset):
    def __init__(self, rootdir):
        self.deceptiveClipsPath = os.path.join(rootdir, 'Deceptive')
        self.truthfulClipsPath = os.path.join(rootdir, 'Truthful')
        self.numTruthful = 10
        self.numDeceptive = 10
        self.deceptiveTensorsPath = os.path.join(rootdir,'DeceptiveFERTensors')
        self.truthfulTensorsPath = os.path.join(rootdir,'TruthfulFERTensors')

    def __len__(self):
        return self.numTruthful+self.numDeceptive

    def __getitem__(self, idx):
      if (idx < self.numTruthful):
        name = "trial_truth_%03d.mp4"%(idx+51)#have to add 1 because 0 indexed and have to add 50 so it starts on right vid
        name2 = os.path.join(self.truthfulTensorsPath, name.replace('mp4','pt'))
        name = os.path.join(self.truthfulClipsPath, name)
        torchclass = torch.tensor([0])

      else:
        name = "trial_lie_%03d.mp4"%((idx-self.numTruthful)+52)#have to add 1 because 0 indexed and have to add 51 so it starts on right vid
        name2 = os.path.join(self.deceptiveTensorsPath, name.replace('mp4','pt'))
        name = os.path.join(self.deceptiveClipsPath, name)
        torchclass = torch.tensor([1])

      #my_clip = mp.VideoFileClip(name)
      #my_audio = my_clip.audio


      unusedvideo,audio,metadata = read_video(name)
      fps = metadata['video_fps']
      audioSampleRate = metadata['audio_fps']

      audio = torchaudio.functional.resample(audio, audioSampleRate, 16000)
      audio = torch.mean(audio, dim=0)#takes mean and gets rid of channels dimesnion
      video = torch.load(name2, map_location=torch.device('cpu'))

      permuted = torch.permute(video, (1,0))#switches embedding and time dimension aka T,F to F,T
      normalizedVideoFeatures = torch.nn.functional.avg_pool1d(permuted, math.ceil(fps/10))#because fps varies grossly across the dataset (10,25,29.97,30 fps) I try to average the features so they are close to 10 fps before I padd them to match the audio
      averagedVideo = torch.permute(normalizedVideoFeatures, (1,0))
      #print(name)
      #video = video.unsqueeze(0)#adding batch dimension #Im actually pretty sure your not supposed to add the batch dimension and instead its automatically added by the dataloader
      #audio = audio.unsqueeze(0)#adding batch dimension

      del normalizedVideoFeatures, permuted, fps, audioSampleRate, unusedvideo
      gc.collect()

      return averagedVideo,audio,torchclass

In [ ]:
class MultimodalBottleneckFusion (nn.Module):
  def __init__(self):
   super(MultimodalBottleneckFusion, self).__init__()
  #Standard transformer encoders use 6 layers (and 6 decoder layers) but the MBT paper uses 12. Should probably use 6 because you already pre-process audio and video
   self.audio_layers = [nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device),
                        nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device),
                        nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device)]

   self.video_layers = [nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device),
                        nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device),
                        nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device), nn.TransformerEncoderLayer(d_model=544, nhead=8).to(device)]

   self.video_layer_0 = nn.TransformerEncoderLayer(d_model=512, nhead=8).to(device) #send rgb FER vectors here before for-loop in order to calculate pairwise self attention

  def __call__(self,video_in, audio_in):
    bottleneck = torch.empty(1,video_in.shape[1],32).normal_(mean=0,std=0.02).to(device) # creates random Gaussian distribution like they did in paper
    #orignally had 50 in above code for dim1 (second dimension)
    curr_video = self.video_layer_0(video_in)
    curr_audio = audio_in
    for i in range(6):

      video_layer_input = torch.cat((curr_video, bottleneck), 2)
      audio_layer_input = torch.cat((curr_audio, bottleneck), 2)

      video_layer_output = self.video_layers[i](video_layer_input.to(device))
      audio_layer_output = self.audio_layers[i](audio_layer_input.to(device))

      curr_video = video_layer_output[:, :, 0:512]
      bottleneck_video_input = video_layer_output[:, :, 512:544]

      curr_audio = audio_layer_output[:, :, 0:512]
      bottleneck_audio_input = audio_layer_output[:, :, 512:544]

      bottleneck = torch.mean(torch.stack((bottleneck_audio_input, bottleneck_video_input)), dim=0)

    del bottleneck
    gc.collect()

    return curr_video, curr_audio


In [ ]:
class Classifier (nn.Module):
  #***putting n for input of first linear layer because I don't know the exact size of it yet
  def __init__(self):
   super(Classifier, self).__init__()
   #self.first_Linear_Layer = nn.Linear(how do i specify the dimension i want to operate on?)#combines the audio and video features of each time-step
   self.first_Linear_Layer = nn.Linear(1024, 1)#takes output of first Linear Layer and compresses each of the temporal 1d vectors to size 1
   #RELU #applies activation function to outputs of second Linear Layer. gets rid of noisy ot uneventful time steps that are negative (assuming 0 is truth)
   ##not sure if i should RELU, or ELU, or Leaky ELU. Which one should I use?
   #self.final_Linear_Layer = nn.Linear()#instead of linear layer do max average or average
   #takes output from RELU activation function and gives final 1,1 logit which is passed into Binary Cross Entropy with built in softmax
   #self.audio_Linear_Layer = nn.Linear(512,1)
   #self.video_Linear_Layer = nn.Linear(512,1)
   #self.combineModes = nn.Linear(2,1) # exactly as the same as just oding 1024,1 on the first linear layer

  def __call__(self,video_input,audio_input):
    bimodal = torch.cat((video_input,audio_input),2)#try hadamard product later 1,50,1024
    condensed = self.first_Linear_Layer(bimodal)
    average = torch.mean(condensed,dim=1)

    del bimodal, condensed
    gc.collect()

    return average







    #audio_output = self.audio_Linear_Layer(audio_input)
    #video_input = self.video_Linear_Layer(video_input)
    #bimodal = torch.cat((video_input,audio_output),2)
    #combinedmodes = self.combineModes(bimodal)
    #probability = torch.mean(combinedmodes,dim=1)
    #signoidProbability = torch.signoid()







In [ ]:
class DeceptionModel (nn.Module):
  def __init__(self):
   super(DeceptionModel, self).__init__()
   self.classifer = Classifier()
   self.transformerEncoder = MultimodalBottleneckFusion()
   self.bundle = torchaudio.pipelines.WAV2VEC2_BASE
   self.AudioVectorizer = self.bundle.get_model()
   #self.AudioVectorizer = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')#facebook/wav2vec2-base-960h facebook/wav2vec2-base
   #self.AudioVectorizer = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
   self.AudioVectorizer.requires_grad = True
   self.videoCompression = nn.Linear(1408,512)
   self.audioCompression = nn.Linear(768,512)

  def __call__ (self, FERvideo, audio): #shape of audio is B C(channels) T(16000 * duration in seconds) shape of my video is B T H W C (actually since this is the video feautres shape is B T F)
    #print(FERvideo.shape)
    #print(audio.shape)
    use_cuda = torch.cuda.is_available()
    device = 'cuda' if use_cuda else 'cpu'
    #print(device)
    #FERvideo, audio = FERvideo.to(device), audio.to(device)#tensors have to be on GPU because Wav2Vec2 model weights are on gpu



    #print(torch.argmax(audio,dim))
    #vectorizedAudio = self.AudioVectorizer(audio, False, None, False, None, None, 'pt', 16000)#.last_hidden_state
    #vectorizedAudio = self.AudioVectorizer(audio).last_hidden_state
    vectorizedAudioList = self.AudioVectorizer.extract_features(audio)
    del audio
    gc.collect()

    ## vectorizedAudioList is a list of list of tensors, so the below code is turning it into a tensor. Actually dim of first list is only 1 so its just list of tensors
    # each tensor in list is formatted like so: `(batch, time frame, feature dimension)`
    # the batch dimension was 1 because I only passed in 1 input, and it will continue like that until I switch batch size away from 1
    # however, there are 12 different elements of the array. they are all the same size and the batch dimension was already included in each individual tensor
    # so I think it is just 12 different features the thing picked up on. It could be that it cut the sound into 12 time frames, but I doubt it since the time dim
    # seems to roughly match the 50 sample rate * duration of vid in seconds

    vectorizedAudioTensor = vectorizedAudioList[0][0]
    vectorizedAudioTensor = torch.unsqueeze(vectorizedAudioTensor, 0)

    for i in range (len(vectorizedAudioList[0])-1):

      vectorizedAudioTensor = torch.cat((vectorizedAudioTensor, torch.unsqueeze(vectorizedAudioList[0][i+1],0)), 0)

    del vectorizedAudioList
    gc.collect()
    # we added all the elements in the array across a new dimension by unsqueezing to preserve the batch size. It should now be 12, B, T, F

    #for i in range (len(vectorizedAudioList)-1):
    #  print(i)
    #  tempTensor = torch.unsqueeze(vectorizedAudioList[i+1][0], 0)
    #  tempTensor = torch.unsqueeze(tempTensor, 0)
    #  for j in range (len(vectorizedAudioList[i+1])-1):
    #    tempTensor = torch.cat((tempTensor, torch.unsqueeze(torch.unsqueeze(vectorizedAudioList[i+1][j+1],0),0)), 1)
    #  vectorizedAudioTensor = torch.cat((vectorizedAudioTensor, tempTensor),0)

    vectorizedAudioTensor = torch.mean(vectorizedAudioTensor, dim=0) # averages 12 transformer encoder heads outputted by wav2vec

    #vectorizedAudio = torch.Tensor(vectorizedAudio)

    paddedAudio = torch.cat((vectorizedAudioTensor, torch.zeros((vectorizedAudioTensor.shape[0], 1, vectorizedAudioTensor.shape[2])).to(device)), 1) #fixes audios so 50 samples per second instead of 50*seconds -1
    permuted = torch.permute(paddedAudio, (0,2,1))#switches embedding ant time dimension aka switches B T F to B F T
    condensedAudioVecs = torch.nn.functional.avg_pool1d(permuted, 5)
    averagedAudio = torch.permute(condensedAudioVecs, (0,2,1))#now this averaged audio is BTF (f = 768) where T is 10 FPS
    transformerInputAudio = self.audioCompression(averagedAudio)

    #pass it through FER feature extractors
    #FERfeatures = self.videoEmbeddingExtraction(video)
    #print(FERfeatures.shape)
    #breakpoint
    transformerInputVideo = self.videoCompression(FERvideo)

    # padding video and audio to be same size
    B,TimeVideo,F = transformerInputVideo.shape
    B,TimeAudio,F = transformerInputAudio.shape

    if (TimeVideo > TimeAudio):
      transformerInputAudio = torch.cat((transformerInputAudio, torch.zeros((transformerInputAudio.shape[0], TimeVideo-TimeAudio, transformerInputAudio.shape[2])).to(device)), 1)

    elif (TimeAudio > TimeVideo):
      transformerInputVideo = torch.cat((transformerInputVideo, torch.zeros((transformerInputVideo.shape[0], TimeAudio-TimeVideo, transformerInputVideo.shape[2])).to(device)), 1)




    videoEmbed, audioEmbed = self.transformerEncoder(transformerInputVideo.to(device), transformerInputAudio.to(device))
    FinalLogitTruthFalse = self.classifer(videoEmbed, audioEmbed) # 0 for truth 1 for lie

    del paddedAudio,permuted, condensedAudioVecs, averagedAudio, transformerInputAudio, transformerInputVideo, videoEmbed, audioEmbed, vectorizedAudioTensor, B, TimeVideo, TimeAudio, F
    gc.collect()
    return FinalLogitTruthFalse

#  def videoEmbeddingExtraction (self, video): # manually implementing some of the stuff from HSME
#    #video dimensions are B T H W C
#    # dimensions for each frame (which extract face requires) B'(B*T) H W C
#    B,T,H,W,C = video.shape
#    #cv2_imshow(torch.squeeze(torch.squeeze(video[0,0,:,:,:])).cpu().numpy())
#    mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
#    def extract_face(frame):
#        bounding_boxes, probs = mtcnn.detect(frame, landmarks=False)
#        print(bounding_boxes.shape)
#        #bounding_boxes = bounding_boxes.astype(np.float64)
#        #bounding_box= torch.take_along_dim(torch.from_numpy(bounding_boxes), torch.from_numpy(np.argmax(probs,1)),1)
#        bounding_boxes2 = []
#        #bounding_boxes = np.vstack(bounding_boxes[:]).astype(np.float64)
#        #print(bounding_boxes.shape)
#        #bounding_boxes = bounding_boxes.astype(np.float64)
#        for i in range (bounding_boxes.shape[0]):
#          indexOfBestFace = np.argmax(probs[i])
#          arrayOfBoundingBoxesForOneFrame = bounding_boxes[i]
#          bounding_boxes2.append(arrayOfBoundingBoxesForOneFrame[indexOfBestFace])#originalbounding_boxes2.append(bounding_boxes[i, indexOfBestFact,:])
#
#        del indexOfBestFace
#        gc.collect()
#
#        del arrayOfBoundingBoxesForOneFrame
#        gc.collect()

#        del bounding_boxes
#        gc.collect()
#
#        bounding_boxes2 = np.stack(bounding_boxes2)
#        bounding_boxes2 = torch.from_numpy(bounding_boxes2.astype('int32'))#.int()
#        #box = bounding_box.int()
#        batchlist = []
#        i = 0
#        height = 0
#        width = 0
#        for batch in range(bounding_boxes2.shape[0]):
#          i += 1
#          x1,y1,x2,y2=bounding_boxes2[batch,0:4]
#          face_img=frame[batch, y1:y2,x1:x2,:] # batch, h, w, c
#          batchlist.append(face_img)
#          height += face_img.shape[0]
#          width += face_img.shape[1]

#        averageHeigth = int(height/i)
#        averageWidth = int(width/i)

#        del bounding_boxes2
#        gc.collect()
#
#        print(averageHeigth)
#        print(averageWidth)
#        resize = transforms.Compose([transforms.Resize((averageHeigth,averageWidth),interpolation=InterpolationMode.BILINEAR, max_size=None, antialias=True)])

#        for j in range(i):#i should be equal to bounding_boxes2.shape[0]
#          batchlist[j] = resize(torch.permute(batchlist[j], (2,0,1)))#swithcing from hwc to chw and then resizing image

#        return torch.stack(batchlist)

#    facialRecognitionBatch  = video.reshape(B*T, H, W, C) #got rid of time dimension from input
#    del video
#    gc.collect()
    #print(facialRecognitionBatch.shape)
    #permuted = torch.permute(facialRecognitionBatch, (0, 2,3,1))# orignally in C H W for pytorch, after permute in H W C for openCV
    #cv2_imshow(torch.squeeze(facialRecognitionBatch[0,:,:,:]).cpu().numpy())

    #facialRecognitionBatchNumpy = facialRecognitionBatch.cpu().numpy()
    #facialRecognitionBatchNumpyBGR = facialRecognitionBatchNumpy[:,:,:,::-1]
    #tensorBatch = torch.from_numpy(facialRecognitionBatchNumpyBGR)
    #red, green, blue = torch.split(facialRecognitionBatch,3,3)#converting to BGR for open cv
    #facialRecognitionBatch_BGR = torch.cat(3, [blue,green,red])
    #cv2_imshow(torch.squeeze(facialRecognitionBatch_BGR[0,:,:,:]).cpu().numpy())
#    face = extract_face(facialRecognitionBatch) # B C H W

#    del facialRecognitionBatch
#    gc.collect()

#    torch.cuda.empty_cache()

#    print("face.shape")
#    print(face.shape)
#    model_name='enet_b2_8'
#    fer=HSEmotionRecognizer(model_name=model_name,device=device)
    #features=fer.extract_features(face)
    #print(features)

#    test_transforms = transforms.Compose(
#                [z
#                    transforms.Resize((fer.img_size,fer.img_size)),
#                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                                std=[0.229, 0.224, 0.225])
#                ]
#            )

#    #face = torch.permute(face, (0,3,1,2))
#    img_tensor = test_transforms(face.double())
#
#    del face
#    gc.collect()

#    features = fer.model(img_tensor.to(fer.device).float())
#    features = features.reshape(B,T,-1)
#    return features

In [ ]:
dataset = CustomDeceptionDataset('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips')
# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)#put batchsize to 1 for now only because videos are diff lenghts
#after make videos the same lenght will swithc batch size back to 4 and

In [ ]:

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
mainModel = DeceptionModel()
mainModel.to(device)#what is cuda? should i use that?
lossFunction = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(mainModel.parameters(), lr=0.001)#orinally had momentum=0.9 as a paramater but didnt work so scraped it
##instantiate vide model
##



Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960.pth
100%|██████████| 360M/360M [00:03<00:00, 95.4MB/s]


In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        video, audio, labels = data
        video, audio, labels = video.to(device), audio.to(device), labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()
        #video = video[:,0:300,:]
        #audio = audio[:,0:500000]
        # Make predictions for this batch
        outputs = mainModel(video, audio)

        # Compute the loss and its gradients

        loss = lossFunction(outputs, labels.float())
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
        del video, audio, labels, outputs
        gc.collect()
        torch.cuda.empty_cache()
        #print(torch.cuda.memory_summary())

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    use_cuda = torch.cuda.is_available() # this and the two lines after are just meant to make sure its utilizer gpu
    device = 'cuda' if use_cuda else 'cpu'
    mainModel.to(device)
    # Make sure gradient tracking is on, and do a pass over the data
    mainModel.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    #running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    #model.eval()

    # Disable gradient computation and reduce memory consumption.
    #with torch.no_grad():
    #    for i, vdata in enumerate(validation_loader):
    #        vinputs, vlabels = vdata
    #        voutputs = model(vinputs)
    #        vloss = loss_fn(voutputs, vlabels)
    #        running_vloss += vloss
    #
    #avg_vloss = running_vloss / (i + 1)
    print('LOSS train {}'.format(avg_loss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training',
                    { 'Training' : avg_loss},
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    #if avg_vloss < best_vloss:
    #    best_vloss = avg_vloss
    model_path = 'model_{}_{}'.format(timestamp, epoch_number)
    torch.save(mainModel.state_dict(), '/content/drive/MyDrive/ColabNotebooks/'+model_path)# refer to 2:46 Patrick Loeber "Saving and Loading Models" video

    epoch_number += 1

In [ ]:
testingSet = TestDataset('/content/drive/MyDrive/RealLifeDeceptionDetection.2016/Real-life_Deception_Detection_2016/Clips')
# Create data loaders for our datasets; shuffle for training, not for validation
testing_loader = torch.utils.data.DataLoader(testingSet, batch_size=1, shuffle=False)#put batchsize to 1 for now only because videos are diff lenghts
#after make videos the same lenght will swithc batch size back to 4 and

In [ ]:
mainModel.load_state_dict(torch.load("/content/drive/MyDrive/ColabNotebooks/model_20230718_234754_4"))
mainModel.eval()


running_loss = 0.0
correctanswers = 0
truepositive = 0
truenegative = 0
falsenegative = 0
falsepositive = 0

for i, data in enumerate(testing_loader):
        with torch.no_grad():
          # Every data instance is an input + label pair
          video, audio, labels = data
          video, audio, labels = video.to(device), audio.to(device), labels.to(device)

          # Zero your gradients for every batch!

          # Make predictions for this batch
          outputs = mainModel(video, audio)
          print(outputs)

          print(labels.item())

          if (outputs.item() >= 0):
              answer = 1
          else:
              answer = 0
          if (answer == labels.item()):
              correctanswers += 1
              if (answer == 1):
                truepositive += 1
              else:
                truenegative += 1
          else:
              if (answer == 1):
                falsepositive += 1
              else:
                falsenegative += 1



          # Compute the loss and its gradients

          loss = lossFunction(outputs, labels.float())


          # Gather data and report
          running_loss += loss.item()

          del video, audio, labels, outputs
          gc.collect()

print(running_loss/20)

print('Correct answers: {}'.format(correctanswers))
print('True positive: {}'.format(truepositive))
print('True negative: {}'.format(truenegative))
print('False positive: {}'.format(falsepositive))
print('False negative: {}'.format(falsenegative))
print('Accuracy: {}'.format(correctanswers/20))




  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")



tensor([[1.3851]], device='cuda:0')
0
tensor([[1.3601]], device='cuda:0')
0
tensor([[0.9665]], device='cuda:0')
0
tensor([[1.3604]], device='cuda:0')
0
tensor([[1.3034]], device='cuda:0')
0
tensor([[1.0820]], device='cuda:0')
0
tensor([[1.3583]], device='cuda:0')
0
tensor([[1.3849]], device='cuda:0')
0
tensor([[1.3599]], device='cuda:0')
0
tensor([[0.9669]], device='cuda:0')
0
tensor([[1.3416]], device='cuda:0')
1
tensor([[1.2905]], device='cuda:0')
1
tensor([[1.3284]], device='cuda:0')
1
tensor([[1.3362]], device='cuda:0')
1
tensor([[1.3803]], device='cuda:0')
1
tensor([[1.3526]], device='cuda:0')
1
tensor([[1.3493]], device='cuda:0')
1
tensor([[1.3707]], device='cuda:0')
1
tensor([[1.3484]], device='cuda:0')
1
tensor([[1.3206]], device='cuda:0')
1
0.869377176463604
Correct answers: 10
True positive: 10
True negative: 0
False positive: 10
False negative: 0
Accuracy: 0.5
